# Interactive parallel computing with ipyparallel

ipyparallel is a Python package and collection of CLI scripts for controlling clusters for Jupyter.

ipyparallel contains the following CLI scripts:

- ipcluster - start/stop a cluster
- ipcontroller - start a scheduler
- ipengine - start an engine


## Installing/activating ipyparallel

- first install the package: `conda install ipyparallel`
- to be able to launch parallel engines from the dashboard add `c.NotebookApp.server_extensions.append('ipyparallel.nbextension')` to the file `~/.jupyter/jupyter_notebook_config.py`
- then enable the IPython Clusters tab in Jupyter by typing `ipcluster nbextension enable` in a terminal


In [9]:
!conda install ipyparallel

Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/ktw/anaconda2:
#
ipyparallel               5.2.0                    py27_0  


To enable 
`!ipcluster nbextension enable --user`

In [1]:
import numexpr

In [2]:
numexpr.detect_number_of_cores()

4

In [3]:
import numpy as np

In [10]:
from ipyparallel import Client

In [11]:
rc = Client()

In [12]:
rc.ids

[0, 1, 2, 3]

In [3]:
from __future__ import print_function

In [4]:
print("hello")

hello


In [5]:
print "hello"

SyntaxError: invalid syntax (<ipython-input-5-8d90ecae1fe0>, line 1)